In [52]:
import numpy as np
import subprocess
import lumicks.pylake as lk
from skimage.transform import rescale
import os
from pathlib import Path

In [2]:
# Just to test without arguments

movie_path = "data/20240112-113918_WT_0.5nM_15min_po_dodatku_LT_1.tiff"  # Test images
output_path = "output/"


In [3]:
os.makedirs(output_path, exist_ok=True)

In [4]:
movie_filename = os.path.basename(movie_path)

In [46]:
movie = lk.ImageStack(movie_path)  # Loading a stack.
aligned_movie_path=output_path + Path(movie_path).stem + "_aligned.tif"
aligned_movie_filename=Path(movie_path).stem + "_aligned.tif"
movie.export_tiff(
    aligned_movie_path
)  # Save aligned wt stack


In [49]:
# Write Fiji macro to file
correct_drift = True
current_dir = os.getcwd()

current_dir = current_dir.replace('\\','/')
print(current_dir, output_path)
with open("temp_macro.ijm", "w") as f:
    f.write('open("{}");\n'.format(aligned_movie_path))
    f.write('run("Split Channels");\n')
    f.write(('selectImage("C3-{}");\n').format(aligned_movie_filename))
    f.write(
        (
            'run("F4DR Estimate Drift","time=100 max=10 reference=[first frame (default, better for fixed)] apply choose=[{}/{}output.njt]");\n'
        ).format(current_dir, output_path),
    )
    if correct_drift:
        f.write(('selectImage("C2-{}");\n').format(aligned_movie_filename))
        f.write(
            ('run("F4DR Correct Drift", "choose=[{}/{}outputDriftTable.njt]");\n').format(
                current_dir, output_path
            )
        )
        f.write(('selectImage("C1-{}");\n').format(aligned_movie_filename))
        f.write(
            ('run("F4DR Correct Drift", "choose=[{}/{}outputDriftTable.njt]");\n').format(
                current_dir, output_path
            )
        )
        f.write(('selectImage("C3-{}");\n').format(aligned_movie_filename))
        f.write(
            "close();\n",
        )
        f.write(('selectImage("C2-{}");\n').format(aligned_movie_filename))
        f.write(
            "close();\n",
        )
        f.write(('selectImage("C1-{}");\n').format(aligned_movie_filename))
        f.write(
            "close();\n",
        )
        f.write(
            (
                'run("Merge Channels...", "c1=[C1-{} - drift corrected] c2=[C2-{} - drift corrected] c3=[C3-{} - drift corrected] create");\n'
            ).format(aligned_movie_filename, aligned_movie_filename, aligned_movie_filename),
        )
        f.write(('saveAs("Tiff", "{}/{}_drift_corrected.tif");\n').format(current_dir,aligned_movie_path.replace(".tiff","").replace(".tif","")))
        f.write('run("Quit");')

c:/Users/feder/OneDrive/Documents/Slovenija/Kemijski_Institut/Code/walker_tracker output/


In [51]:
path_to_fiji='C:/Program Files/Fiji.app/ImageJ-win64.exe' #How to find automatically?
run_string = '{} -macro "temp_macro.ijm"'.format(path_to_fiji)

#I think the drift correction plugin isn't happy with runnig headless
#run_string = '{} --headless --console -macro "temp_macro.ijm"'.format(path_to_fiji) 

subprocess.run(run_string)

CompletedProcess(args='C:/Program Files/Fiji.app/ImageJ-win64.exe -macro "temp_macro.ijm"', returncode=0)

In [184]:
os.remove("{}/outputDriftTable.njt")    
os.remove("{}/temp_macro.ijm")    
